In [1]:
!pip install transformers
!pip install PyPDF2
!pip install sentencepiece
!pip install nltk

import nltk
nltk.download('stopwords')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
!cp /content/drive/MyDrive/socialpsyhandbook.pdf /content/

In [3]:
file_path = '/content/socialpsyhandbook.pdf'

from PyPDF2 import PdfReader

reader = PdfReader(file_path)

pages = []

for page in reader.pages[10:495]:

  pages.append(str(page.extract_text()))

content = "".join(str(page + '\n') for page in pages)
print(content)

Sosyal Psikolojinin Yüzyılı: Bireyler, Fikirler ve Soruşturmalar: SAGE El Kitabı Sosyal Psikoloji
Yakındaki Harvard Üniversitesi'nde Sosyal İlişkiler (Festinger, 1980). Böylece, Gordon Allport ve Jerome Bruner, savaş 
sonrası dönemin dinamik entelektüel mayasına dahil oldular.
Sadece birkaç yıl içinde, MIT'nin, ana kaygılarının biraz dışında kalan bir çabayı destekleme konusundaki ilgisinin azalması 
nedeniyle, Merkez, Michigan Üniversitesi'ne taşınmaya başladı. Bu geçişin ortasında Kurt Lewin aniden öldü ve halefleri 
liderlik rolünü üstlendi. MIT yıllarına ait başlıca eserler arasında Festinger ve diğerlerinin (1950) savaş sonrası Cambridge, 
Massachusetts'teki konut komplekslerinde üyelik üzerine çalışmaları yer almaktadır. Lewin'in pratik araştırma yapmadaki 
etkisi bir kez daha belirgindi. Bir konut sıkıntısı dikkat gerektiriyordu.
Sırasıyla tutumları ve grup dinamiklerini araştıran Yale ve Michigan'daki araştırma merkezlerine ek olarak, Fritz Heider'in 
yaratıcı çalışmasına yanıt

In [4]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "mrm8488/bert2bert_shared-turkish-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def summarize(content):

  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
 
  input_ids = tokenizer(
      [WHITESPACE_HANDLER(content)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=512
  )["input_ids"]

  output_ids = model.generate(
      input_ids=input_ids,
      max_length=250,
      no_repeat_ngram_size=2,
      num_beams=4
  )[0]

  summary = tokenizer.decode(
      output_ids,
      skip_special_tokens=True,
      clean_up_tokenization_spaces=False
  )

  return summary

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation

stop_words = set(stopwords.words('turkish'))
words = word_tokenize(content)
words = [word for word in words if word not in stop_words]
words = [word for word in words if word not in punctuation]

In [6]:
n = 500
chunks = [words[i:i+n] for i in range(0, len(words), n)]
real_chunks = []

for chunk in chunks:

  chunk = "".join(str(word + ' ') for word in chunk)
  real_chunks.append(chunk)

In [7]:
from tqdm import tqdm

summarized_chunks = []

for chunk in tqdm(real_chunks):

  summarized = summarize(chunk)
  summarized_chunks.append(summarized)

100%|██████████| 435/435 [39:52<00:00,  5.50s/it]


In [8]:
summarized_content = "".join(str(sentence + '\n') for sentence in summarized_chunks)

with open('summarized_book.txt', 'w') as f:
  f.write(summarized_content)